In [1]:
import pandas as pd
import numpy as np
# 计算余弦相似度
from sklearn.metrics.pairwise import cosine_similarity
# 计算欧式距离
from sklearn.metrics import pairwise_distances

In [2]:
# 加载数据
movies = pd.read_csv("movies.csv")
Ratings = pd.read_csv("ratings.csv")
Tags = pd.read_csv("tags.csv")

In [20]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [21]:
Ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [22]:
Tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [116]:
# 用户对已经看过的电影的平均打分情况
Mean = Ratings.groupby(by="userId",as_index=False)['rating'].mean()
Mean.head()

,userId,rating
0,1,4.366379
1,2,3.948276
2,3,2.435897
3,4,3.555556
4,5,3.636364


In [18]:
# 为Ratings添加rating_y（用户的评分电影评分）
Rating_avg = pd.merge(Ratings,Mean,on='userId')
# 添加电影与平均评分的差距
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']
Rating_avg.head()

,userId,movieId,rating_x,timestamp,rating_y,adg_rating
0,1,1,4.0,964982703,4.366379,-0.366379
1,1,3,4.0,964981247,4.366379,-0.366379
2,1,6,4.0,964982224,4.366379,-0.366379
3,1,47,5.0,964983815,4.366379,0.633621
4,1,50,5.0,964982931,4.366379,0.633621


In [25]:
# 透视表，index纵坐标用户ID columns横坐标电影ID 内容，rating_x评分
check = pd.pivot_table(Rating_avg,values='rating_x',index='userId',columns='movieId')
check.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# adg_rating的分布
final = pd.pivot_table(Rating_avg,values='adg_rating',index='userId',columns='movieId')
final.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.366379,NaN,-0.366379,NaN,NaN,-0.366379,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.363636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# 替换透视表中的Nan值
# 替换为用户平均分
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)
# 替换为电影平均分
final_movie = final.fillna(final.mean(axis=0))

In [29]:
final_movie.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.366379,-0.053158,-0.366379,-1.096045,-0.522626,-0.366379,-0.400728,-0.625024,-0.455446,-0.056326,...,-0.205224,-0.705224,0.294776,0.294776,-0.205224,0.294776,-0.205224,-0.205224,-0.205224,0.372024
2,0.312167,-0.053158,-0.234798,-1.096045,-0.522626,0.378461,-0.400728,-0.625024,-0.455446,-0.056326,...,-0.205224,-0.705224,0.294776,0.294776,-0.205224,0.294776,-0.205224,-0.205224,-0.205224,0.372024
3,0.312167,-0.053158,-0.234798,-1.096045,-0.522626,0.378461,-0.400728,-0.625024,-0.455446,-0.056326,...,-0.205224,-0.705224,0.294776,0.294776,-0.205224,0.294776,-0.205224,-0.205224,-0.205224,0.372024
4,0.312167,-0.053158,-0.234798,-1.096045,-0.522626,0.378461,-0.400728,-0.625024,-0.455446,-0.056326,...,-0.205224,-0.705224,0.294776,0.294776,-0.205224,0.294776,-0.205224,-0.205224,-0.205224,0.372024
5,0.363636,-0.053158,-0.234798,-1.096045,-0.522626,0.378461,-0.400728,-0.625024,-0.455446,-0.056326,...,-0.205224,-0.705224,0.294776,0.294776,-0.205224,0.294776,-0.205224,-0.205224,-0.205224,0.372024


In [30]:
final_user.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,-3.663793e-01,1.837611e-16,-3.663793e-01,1.837611e-16,1.837611e-16,-3.663793e-01,1.837611e-16,1.837611e-16,1.837611e-16,1.837611e-16,...,1.837611e-16,1.837611e-16,1.837611e-16,1.837611e-16,1.837611e-16,1.837611e-16,1.837611e-16,1.837611e-16,1.837611e-16,1.837611e-16
2,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,...,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16,2.143879e-16
3,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,...,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16,1.821904e-16
4,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,...,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16,2.055969e-16
5,3.636364e-01,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,...,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16,1.110223e-16


In [40]:
# final_user,final_movie 都是矩阵

# 计算final_user的用户相似度
b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,1.264516e-03,5.525772e-04,0.048419,0.021847,-0.045497,-6.199672e-03,0.047013,1.950985e-02,-8.754088e-03,...,0.018127,-0.017172,-0.015221,-3.705875e-02,-2.912138e-02,0.012016,0.055261,0.075224,-2.571255e-02,0.010932
2,0.001265,0.000000e+00,4.929675e-30,-0.017164,0.021796,-0.021051,-1.111357e-02,-0.048085,1.029826e-29,3.011629e-03,...,-0.050551,-0.031581,-0.001688,1.184549e-30,-4.512223e-30,0.006226,-0.020504,-0.006001,-6.009100e-02,0.024999
3,0.000553,4.929675e-30,0.000000e+00,-0.011260,-0.031539,0.004800,4.872283e-31,-0.032471,2.324185e-30,8.830513e-31,...,-0.004904,-0.016117,0.017749,2.860060e-31,-1.430628e-03,-0.037289,-0.007789,-0.013001,4.346619e-30,0.019550
4,0.048419,-1.716402e-02,-1.125978e-02,0.000000,-0.029620,0.013956,5.809139e-02,0.002065,-5.873603e-03,5.159032e-02,...,-0.037687,0.063122,0.027640,-1.378212e-02,4.003747e-02,0.020590,0.014628,-0.037569,-1.788358e-02,-0.000995
5,0.021847,2.179571e-02,-3.153892e-02,-0.029620,0.000000,0.009111,1.011715e-02,-0.012284,2.745189e-30,-3.316512e-02,...,0.015964,0.012427,0.027076,1.246135e-02,-3.627206e-02,0.026319,0.031896,-0.001751,9.382892e-02,-0.000278


In [47]:
# 计算final_movie的用户相似度
cosine = cosine_similarity(final_movie)
np.fill_diagonal(cosine, 0 )
similarity_with_movie = pd.DataFrame(cosine,index=final_movie.index)
similarity_with_movie.columns=final_user.index
similarity_with_movie.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.988283,0.978406,0.964220,0.986819,0.970456,0.971643,0.987468,0.986382,0.973397,...,0.987335,0.978916,0.917922,0.983978,0.978638,0.959693,0.976120,0.932806,0.989380,0.952774
2,0.988283,0.000000,0.987141,0.971166,0.995793,0.979893,0.981852,0.995168,0.995108,0.981285,...,0.996067,0.988455,0.929086,0.993014,0.988206,0.968868,0.983619,0.940224,0.997957,0.963114
3,0.978406,0.987141,0.000000,0.961237,0.985179,0.970773,0.971932,0.985140,0.985263,0.971464,...,0.986072,0.978562,0.921433,0.983193,0.978363,0.957067,0.974114,0.930653,0.988086,0.954265
4,0.964220,0.971166,0.961237,0.000000,0.968638,0.955187,0.958876,0.970090,0.969158,0.959626,...,0.970625,0.964815,0.903118,0.967106,0.963962,0.942701,0.958891,0.911591,0.972100,0.935866
5,0.986819,0.995793,0.985179,0.968638,0.000000,0.978368,0.980011,0.992905,0.993494,0.979161,...,0.994448,0.986028,0.928126,0.991066,0.986090,0.967230,0.982366,0.938353,0.996584,0.960854


In [48]:
# 寻找最近的n个邻居
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [49]:
# 对于final_user的最近三十个
sim_user_30_u = find_n_neighbours(similarity_with_user,30)
sim_user_30_u.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,...,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userId,,,,,,,,,,,,,,,,,,,,,
1,301,597,414,477,57,369,206,535,590,418,...,484,469,72,593,44,297,434,483,449,552
2,189,246,378,209,227,326,393,332,196,528,...,114,153,596,495,407,567,93,308,496,222
3,441,496,549,231,527,537,313,518,244,246,...,309,586,230,303,34,161,39,47,127,138
4,75,137,590,391,43,128,462,250,290,85,...,472,593,299,32,349,268,159,199,173,426
5,145,35,565,134,58,444,446,347,530,142,...,94,569,411,588,584,404,498,323,536,437


In [50]:
# 对于final_movie的最近三十个
sim_user_30_m = find_n_neighbours(similarity_with_movie,30)
sim_user_30_m.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,...,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userId,,,,,,,,,,,,,,,,,,,,,
1,49,54,72,53,515,550,189,133,26,513,...,145,60,595,574,556,92,435,163,439,340
2,49,189,515,53,25,145,54,26,87,194,...,65,163,251,150,299,440,519,507,245,521
3,515,49,25,53,496,54,442,72,26,87,...,92,595,556,65,463,251,180,150,163,439
4,581,54,189,49,25,515,300,53,472,251,...,26,289,463,609,214,206,507,574,120,556
5,145,49,515,53,609,26,189,550,35,87,...,293,142,72,92,556,439,478,163,595,299


In [52]:
# 获得两个用户共同评价电影矩阵
def get_user_similar_movies( user1, user2 ):
    common_movies = Rating_avg[Rating_avg.userId == user1].merge(
    Rating_avg[Rating_avg.userId == user2],
    on = "movieId",
    how = "inner" )
    return common_movies.merge( movies, on = 'movieId' )

In [54]:
a = get_user_similar_movies(1,49)
a = a.loc[ : , ['rating_x_x','rating_x_y','title']]
a

,rating_x_x,rating_x_y,title
0,4.0,4.0,Braveheart (1995)
1,4.0,4.0,Forrest Gump (1994)
2,5.0,4.5,Schindler's List (1993)
3,5.0,4.5,E.T. the Extra-Terrestrial (1982)
4,4.0,4.0,Alien (1979)
5,4.0,4.5,Saving Private Ryan (1998)
6,5.0,4.5,"Matrix, The (1999)"


In [88]:
# 得到用户基于相似用户对item的评分
def User_item_score(user,item):
    # 获得user相似的三十个用户
    a = sim_user_30_m[sim_user_30_m.index==user].values
    # a 转变成列表
    b = a.squeeze().tolist()
    # 获得user-item 电影平均分矩阵某项电影的全部用户评分
    c = final_movie.loc[:,item]
    # 获得c中在相似用户里面的值
    d = c[c.index.isin(b)]
    # 此用户相似用户的对此电影的偏差喜好评分
    f = d[d.notnull()]
    # 获得此用户的电影平均分
    avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
    # 相似用户ID列表
    index = f.index.values.squeeze().tolist()
    # 此用户对于相似用户的相似程度列表
    corr = similarity_with_movie.loc[user,index]
    # 生成用户对此电影的偏差评分和对用户的相似度
    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    # fin添加一列预测偏差喜欢评分score
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    
    # 计算公式 所有score和／所有相关系数和 + 平均打分
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score

In [90]:
# 一号用户对1号电影的评分情况
score = User_item_score(1,1)
print("score (u,i) is",score)

score (u,i) is 4.69412455108947


In [92]:
Rating_avg

,userId,movieId,rating_x,timestamp,rating_y,adg_rating
0,1,1,4.0,964982703,4.366379,-0.366379
1,1,3,4.0,964981247,4.366379,-0.366379
2,1,6,4.0,964982224,4.366379,-0.366379
3,1,47,5.0,964983815,4.366379,0.633621
4,1,50,5.0,964982931,4.366379,0.633621
...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,3.688556,0.311444
100832,610,168248,5.0,1493850091,3.688556,1.311444
100833,610,168250,5.0,1494273047,3.688556,1.311444
100834,610,168252,5.0,1493846352,3.688556,1.311444


In [98]:
# 将Rating_avg中的movieID修改类型为str
Rating_avg = Rating_avg.astype({"movieId": str})
# 获得Rating列表中 用户ID观看的所有movieID
Movie_user = Rating_avg.groupby(by = 'userId')['movieId'].apply(lambda x:','.join(x))
Movie_user

userId
1      1,3,6,47,50,70,101,110,151,157,163,216,223,231...
2      318,333,1704,3578,6874,8798,46970,48516,58559,...
3      31,527,647,688,720,849,914,1093,1124,1263,1272...
4      21,32,45,47,52,58,106,125,126,162,171,176,190,...
5      1,21,34,36,39,50,58,110,150,153,232,247,253,26...
                             ...                        
606    1,7,11,15,17,18,19,28,29,32,36,46,47,50,58,68,...
607    1,11,25,34,36,86,110,112,150,153,165,188,204,2...
608    1,2,3,10,16,19,21,24,31,32,34,39,44,47,48,50,6...
609    1,10,110,116,137,150,161,185,208,231,253,288,2...
610    1,6,16,32,47,50,70,95,110,111,112,153,159,194,...
Name: movieId, Length: 610, dtype: object

In [111]:
Movie_seen_by_user = check.columns[check[check.index==1].notna().any()].tolist()
a = sim_user_30_m[sim_user_30_m.index==1].values
b = a.squeeze().tolist()
d = Movie_user[Movie_user.index.isin(b)]
l = ','.join(d.values)
Movie_seen_by_similar_users = l.split(',')
Movies_under_consideration = list(set(Movie_seen_by_similar_users)-set(list(map(str, Movie_seen_by_user))))
Movies_under_consideration = list(map(int, Movies_under_consideration))

In [112]:
def User_item_score1(user):
    # 用户user看过的movie
    Movie_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    # a为与user相似的三十个用户列表
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    # 最相似三十个用户看过的电影df
    d = Movie_user[Movie_user.index.isin(b)]
    # 所有看过的电影大杂烩
    l = ','.join(d.values)
    # 看过电影的全部列表切分元素
    Movie_seen_by_similar_users = l.split(',')
    # 相似用户看过而自己没有看过的电影列表 第二行将movieid转换成int
    Movies_under_consideration = list(set(Movie_seen_by_similar_users)-set(list(map(str, Movie_seen_by_user))))
    Movies_under_consideration = list(map(int, Movies_under_consideration))
    
    # 没有看过的电影列表评分表
    score = []
    for item in Movies_under_consideration:
        # 获得电影预测评分
        c = final_movie.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_movie.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    # 电影ID 和预测评分 df
    data = pd.DataFrame({'movieId':Movies_under_consideration,'score':score})
    # 获得最高评分五部电影
    top_5_recommendation = data.sort_values(by='score',ascending=False).head(5)
    # 获得最高评分五步电影名称
    Movie_Name = top_5_recommendation.merge(movies, how='inner', on='movieId')
    Movie_Names = Movie_Name.title.values.tolist()
    return Movie_Names

In [113]:
user = int(input("Enter the user id to whom you want to recommend : "))
predicted_movies = User_item_score1(user)
print(" ")
print("The Recommendations for User Id you give")
print("   ")
for i in predicted_movies:
    print(i)

Enter the user id to whom you want to recommend : 1
 
The Recommendations for User Id you give
   
Three Billboards Outside Ebbing, Missouri (2017)
Return of Martin Guerre, The (Retour de Martin Guerre, Le) (1982)
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)
Godfather, The (1972)
Shawshank Redemption, The (1994)
